In [181]:
# Set up imports
import pandas as pd
import numpy as np
import re
import sys

In [182]:
# Load datasets
ohio_df = pd.read_csv(r"Unstandardized Data\Ohio_completed.csv")
ma_df = pd.read_csv(r"Unstandardized Data\MA_completed.csv")

In [183]:
# Delete extra columns
ohio_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 7', 'Filename','Legal'], axis=1, inplace=True)

In [184]:
# Look and check through MA Data
ma_df.head()
ma_df.shape

(382, 14)

In [185]:
# Delete extra columns
ma_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1','URL', 'Filename','Conduct'], axis=1, inplace=True)

In [186]:
ma_df.reset_index(drop=True)
ma_df.shape
ma_df.head()


,Unnamed: 0.1.1.1,Date,Year,Enforcement Action,Entity Name,Entity Type,Location,Text
0,0,8/29/2021,2003.0,Temporary Cease and Desist,Mortgage One Financial Corp,Mortgage Broker,Norwood MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
1,1,10/31/2021,2003.0,Temporary Cease and Desist,Platinum Capital Group,"Mortgage Lender,Mortgage Broker","HQ Irvine CA, branch located in Concord CA","COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
2,2,10/31/2021,2003.0,Temporary Cease and Desist,"KN Management, Inc.",Mortgage Broker,Concord MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
3,3,12/23/2021,2003.0,Temporary Cease and Desist,INSTAFI.COM,"Mortgage Lender,Mortgage Broker",Irvine CA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
4,4,12/30/2021,2003.0,Consent Order,"NovaStar Home Mortgage, Inc.","Mortgage Lender,Mortgage Broker",Kansas City MO,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."


In [187]:
ma_df = ma_df.loc[pd.notna(ma_df['Text'])]
ma_df.head()
#missing_df = missing_df[['Filename']]
#missing_df.to_csv("Ohio_missing.csv")

,Unnamed: 0.1.1.1,Date,Year,Enforcement Action,Entity Name,Entity Type,Location,Text
0,0,8/29/2021,2003.0,Temporary Cease and Desist,Mortgage One Financial Corp,Mortgage Broker,Norwood MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
1,1,10/31/2021,2003.0,Temporary Cease and Desist,Platinum Capital Group,"Mortgage Lender,Mortgage Broker","HQ Irvine CA, branch located in Concord CA","COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
2,2,10/31/2021,2003.0,Temporary Cease and Desist,"KN Management, Inc.",Mortgage Broker,Concord MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
3,3,12/23/2021,2003.0,Temporary Cease and Desist,INSTAFI.COM,"Mortgage Lender,Mortgage Broker",Irvine CA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
4,4,12/30/2021,2003.0,Consent Order,"NovaStar Home Mortgage, Inc.","Mortgage Lender,Mortgage Broker",Kansas City MO,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."


In [188]:
# Clean year for Massachusetts
ma_df['Year'] = ma_df['Year'].astype(str).replace(".0","")
ma_df['Year'] = ma_df['Year'].apply(lambda x: x.replace(".0",""))
ma_df['Date'] = ma_df.apply(lambda row: row['Date'].replace("2021",row['Year']), axis=1)
ma_df['Date'] = ma_df.apply(lambda row: row['Date'].replace("2029",row['Year']), axis=1)
ma_df.drop(['Year'], axis=1, inplace=True)
# Standardize Dates
ma_df['Date'] = ma_df.Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
ohio_df['Date'] = ohio_df.Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [189]:
# To upper all text in Massachusetts
ma_df['Enforcement Action'] = ma_df['Enforcement Action'].apply(lambda x: x.upper())
ma_df['Entity Name'] = ma_df['Entity Name'].apply(lambda x: x.upper())
ma_df['Entity Type'] = ma_df['Entity Type'].apply(lambda x: x.upper())
# To upper all text in Ohio as well
ohio_df['Credential'] = ohio_df['Credential'].apply(lambda x: x.upper())
ma_df.head(2)

,Unnamed: 0.1.1.1,Date,Enforcement Action,Entity Name,Entity Type,Location,Text
0,0,08/29/2003,TEMPORARY CEASE AND DESIST,MORTGAGE ONE FINANCIAL CORP,MORTGAGE BROKER,Norwood MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
1,1,10/31/2003,TEMPORARY CEASE AND DESIST,PLATINUM CAPITAL GROUP,"MORTGAGE LENDER,MORTGAGE BROKER","HQ Irvine CA, branch located in Concord CA","COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."


In [190]:
# Standardize Headers
ma_df = ma_df.rename(columns={
    'Location':'City, State'
})
ohio_df = ohio_df.rename(columns={
    'Credential':'Entity Type',
    'Name': 'Entity Name',
    'Action': 'Enforcement Action'
})
# Adding State Label
ma_df['State Enforcing'] = 'MA'
ohio_df['State Enforcing'] = 'OH'

In [191]:
ma_df['Entity Type'].value_counts()

MORTGAGE LENDER,MORTGAGE BROKER                               105
MORTGAGE LOAN ORIGINATOR                                      102
MORTGAGE BROKER                                                64
MORTGAGE COMPANY                                               37
FOREIGN TRANSMITTAL AGENCY                                     16
CHECK CASHER                                                    9
MORTGAGE LENDER                                                 7
DEBT COLLECTOR                                                  6
MOTOR VEHICLE SALES FINANCE COMPANY                             5
MORTGAGE BROKER,MORTGAGE LOAN ORIGINATOR                        5
BANK                                                            5
INSURED STATE NONMEMBER BANK                                    2
MORTGAGE BROKER, LENDER,MORTGAGE LOAN ORIGINATOR                1
INSURED STATE NONMEMBER BANK,STATE CHARTERED TRUST COMPANY      1
CREDIT UNION                                                    1
CHECK CASH

In [192]:
# Remove Cases that are not MEAs
ma_df.drop(ma_df[ma_df['Entity Type'].isin(["CHECK CASHER,ELECTRONIC FUND TRANSFER", "CREDIT UNION", "INSURED STATE NONMEMBER BANK", "BANK", "MOTOR VEHICLE SALES FINANCE COMPANY", "CHECK CASHER", "FOREIGN TRANSMITTAL AGENCY"])].index, inplace = True)
# Remove Debt Collectors
ma_df.drop([26,81,99,359,377], inplace = True)

In [193]:
# Rename Ohio Entity Types
ohio_df['Entity Type'] = ohio_df['Entity Type'].replace("OM", "MORTGAGE COMPANY")

In [194]:
# Merge Dataframes
final_df = pd.concat([ohio_df,ma_df])
final_df

,Entity Type,Entity Name,"City, State",Enforcement Action,Date,Text,State Enforcing,Unnamed: 0.1.1.1
0,RM,"UNITED MORTGAGE GROUP, INC.","PARMA, OH",REPORT AND RECOMMENDATION,01/27/2000,NaN,OH,NaN
1,RM,"SUPERIOR HOME MORTGAGE, LLC","EUCLID, OH",NOTICE OF INTENT TO DENY,04/12/2000,NaN,OH,NaN
2,RM,"UNITED MORTGAGE GROUP, INC.","PARMA, OH",DIVISION ORDER,04/12/2000,NaN,OH,NaN
3,RM,"SUPERIOR HOME MORTGAGE, LLC","EUCLID, OH",REPORT AND RECOMMENDATION,11/02/2000,NaN,OH,NaN
4,RM,"T. P. O. T. T., INC.","COLUMBUS, OH",NOTICE OF INTENT TO REVOKE,04/18/2001,STATE OF OHIO\nDEPARTMENT OF COMMERCE\nDIVISIO...,OH,NaN
...,...,...,...,...,...,...,...,...
374,MORTGAGE LOAN ORIGINATOR,JAMES P. DEORIO�,NaN,CONSENT ORDER,11/26/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,374.0
375,MORTGAGE LOAN ORIGINATOR,MATTHEW J. VANCE,NaN,CONSENT ORDER,11/30/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,375.0
376,MORTGAGE LOAN ORIGINATOR,RUSSELL TUNICK,NaN,CONSENT ORDER,12/02/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,376.0
378,MORTGAGE COMPANY,"ADVANCED MORTGAGE SERVICES, LLC",NaN,CONSENT ORDER,12/13/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,378.0


In [197]:
# Remove newlines and spaces
def remove_newlines(text):
    if not pd.isna(text):
        text= re.sub(r"(\n( ?))+", "\n", text)
        text = re.sub(r"   +", "  ", text)
        text = text.replace("\n"," ")
    return text
final_df['Text'] = final_df['Text'].apply(remove_newlines)

# Calculate document length
def calc_length(text):
    if not pd.isna(text):
        return len(text)
    return 0
final_df['Text Length'] = final_df['Text'].apply(calc_length)

# Remove all quotation marks from text
def remove_quotes(text):
    if not pd.isna(text):
        return text.replace('"',"")
    return text
final_df['Text'] = final_df['Text'].apply(remove_quotes)
final_df.drop(columns="Unnamed: 0.1.1.1", axis=1, inplace=True)

KeyError: "['Unnamed: 0.1.1.1'] not found in axis"

In [1]:
# Function to extract text between two strings (CAPS SENSITIVE)
def extract_text(text, sub1, sub2):
    index_1 = text.index(sub1)
    index_2 = text.index(sub2)
    return text[index_1 + len(sub1): index_2]

test_string = "COMMONWEALTH OF MASSACHUSETTS  Suffolk, SS.  COMMISSIONER OF BANKS  MORTGAGE BROKER  LICENSING  Docket No. 2010-224  FINDINGS OF FACT AND  TEMPORARY ORDER TO  CEASE AND DESIST  In the Matter of  LEADPOINT, INC.  Los Angeles, California  Mortgage Broker License No(s). MB4077  The Commissioner of Banks (Commissioner) having determined that LEADPOINT, INC. (LEADPOINT or the Licensee), with its main address located at 11661 San Vincente Boulevard, Suite 600, Los Angles, California, 90049, has engaged in, or is engaging in, or is about to engage in, acts or practices constituting violations of Massachusetts General Laws chapter 255E and applicable regulations found at 209 CMR 42.00  et seq. , hereby issues the following FINDINGS OF FACT AND TEMPORARY ORDER TO CEASE AND DESIST (Temporary Order) pursuant to General Laws chapter 255E, section 7(b) and chapter 255F, section 11(a).  FINDINGS OF FACT  The Division of Banks (Division), through the Commissioner, has jurisdiction over the licensing and regulation of persons and entities engaged in the business of a mortgage lender and mortgage broker in Massachusetts pursuant to Massachusetts General Laws chapter 255E, section 2.  The Division, through the Commissioner, also has jurisdiction over the licensing and regulation of persons engaged in the business of a mortgage loan originator in Massachusetts pursuant to Massachusetts General Laws chapter 255F, section 2.  LEADPOINT is, and at all relevant times has been, a Massachusetts licensed mortgage broker doing business in the Commonwealth.  A. Failure to Submit Financial Statements as Required  The Division's regulation 209 CMR 42.06(2)(b)3 states:  Renewal Applications. Within 90 days of the close of its fiscal year, an Applicant for [mortgage broker] license renewal shall submit financial statements for the preceding fiscal year that have been  audited or reviewed  by an independent certified public accountant and such other financial information as the Commissioner may require. [Emphasis supplied.]  On or about October 22, 2009, the Division sent detailed instructions to all licensed mortgage brokers and mortgage lenders regarding the 2010 license renewal application process.  The license renewal instructions notified each recipient that by December 31, 2009, all mortgage lender and mortgage broker licensees were required to remain in compliance with 209 CMR 42.03 and 209 CMR 42.06, respectively, including the requirement to submit to the Division audited or reviewed financial statements, as determined by the type of license(s) held, within ninety (90) days of the end of a licensee's fiscal year end.  On or about March 16, 2010, the Division sent a reminder notice to all licensed mortgage brokers and mortgage lenders regarding the 2010 license renewal requirements. Pursuant to the instructions contained therein, all licensed mortgage brokers and mortgage lenders with a fiscal year end date of December 31  st  were directed to file through the Nationwide Mortgage Licensing System (NMLS) the required financial statements by no later than March 31, 2010.  On April 12, 2010, LEADPOINT uploaded draft financial statements through the NMLS. These financial statements were not in accordance with the 2010 license renewal instructions or the Division's regulation, as they were neither audited nor reviewed by an independent certified public accountant, as required.  As of the date of this Temporary Order, the Licensee has failed to provide the required financial statements to fulfill the Division's license renewal requirements.  On December 30, 2009, LEADPOINT, through a duly appointed and authorized representative, affirmed a series of attestations in completing LEADPOINT's Massachusetts license renewal application through the NMLS (the Renewal Attestation). The Renewal Attestation provided that the licensee acknowledged and agreed to expediently update and correct the information in the licensee's NMLS record as it changed. LEADPOINT further attested, in part, to an understanding that, submitting false or misleading information, or omitting pertinent or material information, may be grounds for administrative action and/or criminal action.  LEADPOINT's Renewal Attestation further provided, in part, the following:  The Licensee/Registrant meets the financial responsibility requirements and/or net worth requirements, as required by each jurisdiction being requested to renew its license/registration; and  The licensee acknowledged an understanding, and agreed to comply with, the laws and regulations pertaining to the conduct of the licensed business.  On or about May 13, 2010, the Division issued a Final Notice to LEADPOINT, providing notification of the deficiencies with LEADPOINT's license renewal status and directing LEADPOINT to comply with the license renewal process by supplying the outstanding renewal requirements identified in paragraph(s) 4 through 11. The directive required LEADPOINT to respond with all applicable information by no later than May 24, 2010. As of the date of this Temporary Order, LEADPOINT has failed to submit all, or part of, the required documentation to the Division.  CONCLUSIONS OF LAW  Based upon the information contained in Paragraphs 1 through 12, by failing to file the financial statements that are required pursuant to the license renewal process, LEADPOINT has violated 209 CMR 42.06(2)(b)3.  Based upon the information contained in Paragraphs 1 through 12, the Commissioner has determined that:  LEADPOINT has engaged in, is engaging in, or is about to engage in, acts or practices which warrant the belief that it is not operating honestly, fairly, soundly and efficiently in the public interest in violation of standards governing the licensing and conduct of a mortgage lender and mortgage broker including, but not limited to, the provisions of the Division's regulations at 209 CMR 42.00 et seq.; and  The public interest will be irreparably harmed by delay in issuing an ORDER TO CEASE AND DESIST to LEADPOINT.  Based upon the information contained in Paragraphs 1 through 12, had the facts and conditions found therein existed at the time of LEADPOINT'S original mortgage broker license application, the Commissioner would have been warranted in refusing to issue such license(s). Further, the facts and conditions set forth in Paragraphs 1 through 12 present sufficient grounds for the revocation of LEADPOINT'S mortgage broker license(s) pursuant to Massachusetts General Laws chapter 255E, section 6 and the Division's regulation at 209 CMR 42.07(2)(b).  ORDER TO CEASE AND DESIST  After taking into consideration the FINDINGS OF FACT and CONCLUSIONS OF LAW stated herein, it is hereby:  ORDERED that LEADPOINT and any and all officers, members, managers, employees, independent contractors, or agents, operating on behalf of LEADPOINT, and their successors or assigns, shall immediately cease engaging in the activities of: (a) mortgage broker, as those activities are defined under Massachusetts General Laws chapter 255E, section 1, and (b) a mortgage loan originator, as those activities are defined under Massachusetts General Laws chapter 255F, section 1, on behalf of LEADPOINT relative to any residential property in Massachusetts, not otherwise expressly permitted by the terms of this Temporary Order. Therefore, LEADPOINT is ordered immediately to cease soliciting or accepting, either directly or indirectly, any residential mortgage loan applications from consumers for residential property located in Massachusetts.  IT IS FURTHER ORDERED that LEADPOINT, shall immediately place any fees previously collected from Massachusetts consumers relative to any pending mortgage loan applications in a separate escrow account maintained at a federally insured bank.  IT IS FURTHER ORDERED that LEADPOINT shall immediately place with one or more qualified broker(s) or lender(s), as appropriate based on the status of the application and with no loss to applicants, all of its pending Massachusetts residential mortgage loan applications. LEADPOINT shall place such applications with a licensed Massachusetts broker(s) or lender(s), as appropriate, or with a financial institution(s) that is exempt from the mortgage licensing requirements under Massachusetts General Laws chapter 255E, section 2.  IT IS FURTHER ORDERED that as soon as possible, but in no event later than two days after the effective date of this Temporary Order, LEADPOINT shall submit the following information in writing to the Commissioner:  A detailed record of all pending residential mortgage loan applications, which shall include, but is not limited to, the following: customer name, address, telephone number; all prepaid loan fees submitted by the customer; amount of loan; application status (i.e. filed, submitted to lenders); scheduled closing date; rate lock status; the location of all original open application files; and a list of applicable wholesale lenders. The latter list should include telephone numbers of contact persons familiar with LEADPOINT's submitted loans.  IT IS FURTHER ORDERED that LEADPOINT shall immediately secure all pending mortgage loan application files and, to the extent that any original documents must be forwarded to the relevant mortgage lender or mortgage broker pursuant to this Temporary Order, a copy of such document, correspondence, or paper relating to the mortgage loan shall be retained in LEADPOINT's books and records and shall be available to the Commissioner, in their entirety, immediately upon request on the date and time specified by the Commissioner.  IT IS FURTHER ORDERED that LEADPOINT shall submit financial statements for the preceding fiscal year in the form required by the license renewal provisions of 209 CMR 42.00  et seq .  IT IS FURTHER ORDERED that this Temporary Order shall become effective immediately and shall remain in effect unless vacated, modified, or suspended by the Commissioner or upon court order after review under Massachusetts General Laws chapter 30A.  It is understood that the Licensee may petition the Commissioner to vacate or modify this Temporary Order upon presenting to the Division satisfactory evidence that the Licensee has fully complied with the provisions of this Temporary Order.  IT IS FURTHER ORDERED that a hearing will be scheduled on this matter to determine whether or not such Temporary Order shall become permanent and final only upon receipt of a written request for such a hearing from LEADPOINT within twenty (20) days of the effective date of this Temporary Order. If no hearing is requested within this twenty (20) day period, this Temporary Order shall become permanent and final until it is modified or vacated by the Commissioner.  BY ORDER AND DIRECTION OF THE COMMISSIONER OF BANKS.  Dated at Boston, Massachusetts, this 15th day of June, 2010  By: Steven L. Antonakes  Commissioner of Banks  Commonwealth of Massachusetts  "
sub1 = ".  FINDINGS OF FACT"
sub2 = "CONCLUSION"

print(extract_text(test_string, sub1, sub2))

# ohio_df["Text"] = ohio_df['Text'].apply(lambda x: extract_text(x, "alleged that: A."))

  The Division of Banks (Division), through the Commissioner, has jurisdiction over the licensing and regulation of persons and entities engaged in the business of a mortgage lender and mortgage broker in Massachusetts pursuant to Massachusetts General Laws chapter 255E, section 2.  The Division, through the Commissioner, also has jurisdiction over the licensing and regulation of persons engaged in the business of a mortgage loan originator in Massachusetts pursuant to Massachusetts General Laws chapter 255F, section 2.  LEADPOINT is, and at all relevant times has been, a Massachusetts licensed mortgage broker doing business in the Commonwealth.  A. Failure to Submit Financial Statements as Required  The Division's regulation 209 CMR 42.06(2)(b)3 states:  Renewal Applications. Within 90 days of the close of its fiscal year, an Applicant for [mortgage broker] license renewal shall submit financial statements for the preceding fiscal year that have been  audited or reviewed  by an indepe

In [196]:
# Output Files
np.set_printoptions(threshold=sys.maxsize)
ohio_df.to_csv(r"Standardized Data\Ohio_completed.csv")
ma_df.to_csv(r"Standardized Data\MA_completed.csv")
final_df.to_csv(r"Standardized Data\Final.csv")